In [3]:
import pandas as pd
import numpy as np

def generate_synthetic_data():
    print("🧪 Iniciando Protocolo de Aumento de Datos (Clinical Data Augmentation)...")
    
    # 1. Cargar datos reales (Base)
    try:
        df_real = pd.read_csv("nhanes_labeled_data.csv")
    except:
        print("❌ Error: Falta 'nhanes_labeled_data.csv'")
        return

    # Columnas necesarias
    cols = ['Sex', 'Age', 'WBC', 'NEU', 'LYM', 'RBC', 'HGB', 'HCT', 'MCV', 'MCH', 'PLT', 'Diagnosis']
    
    # Lista para acumular nuevos pacientes
    synthetic_patients = []
    
    # ==========================================
    # A. GENERADOR SERIE ROJA (ANEMIAS)
    # ==========================================
    
    # 1. Anemia Ferropénica (Microcítica Hipocrómica)
    # Patrón: HGB baja, MCV bajo (<80), MCH bajo (<27)
    print("   -> Sintetizando casos de Anemia Ferropénica...")
    for _ in range(300): # Generar 300 casos
        sex = np.random.choice([1, 2])
        age = np.random.randint(18, 80)
        hgb = np.random.uniform(8.0, 11.5) if sex == 2 else np.random.uniform(9.0, 12.5)
        mcv = np.random.uniform(60.0, 78.0) # Microcitosis
        mch = np.random.uniform(20.0, 26.0) # Hipocromía
        
        # Rellenar resto con valores normales o coherentes
        synthetic_patients.append({
            'Sex': sex, 'Age': age,
            'WBC': np.random.uniform(4.5, 10.0), # Leucos normales
            'NEU': np.random.uniform(2.0, 7.0),
            'LYM': np.random.uniform(1.0, 3.0),
            'RBC': np.random.uniform(3.5, 4.5), # RBC puede ser normal o bajo
            'HGB': hgb, 
            'HCT': hgb * 3, # Regla de tres aproximada
            'MCV': mcv, 'MCH': mch,
            'PLT': np.random.uniform(150, 400),
            'Diagnosis': 'Anemia Ferropénica' # NUEVA ETIQUETA
        })

    # 2. Anemia Megaloblástica (Macrocítica)
    # Patrón: HGB baja, MCV alto (>100)
    print("   -> Sintetizando casos de Anemia Megaloblástica...")
    for _ in range(200):
        hgb = np.random.uniform(7.0, 11.0)
        mcv = np.random.uniform(105.0, 125.0) # Macrocitosis clara
        
        synthetic_patients.append({
            'Sex': np.random.choice([1, 2]), 'Age': np.random.randint(40, 90),
            'WBC': np.random.uniform(3.0, 9.0), # A veces leve leucopenia
            'NEU': np.random.uniform(1.5, 6.0),
            'LYM': np.random.uniform(0.8, 2.5),
            'RBC': np.random.uniform(2.0, 3.5), # RBC bajo
            'HGB': hgb, 'HCT': hgb * 3,
            'MCV': mcv, 
            'MCH': np.random.uniform(28, 35), # Normal o alta
            'PLT': np.random.uniform(100, 300), # A veces trombocitopenia leve
            'Diagnosis': 'Anemia Megaloblástica'
        })

    # ==========================================
    # B. GENERADOR SERIE BLANCA
    # ==========================================
    
    # 3. Leucemia Aguda (Simulación de Patrón Blastos)
    # Patrón: Hiperleucocitosis, Anemia grave, Trombocitopenia grave (Fallo medular)
    print("   -> Sintetizando simulación de Leucemia Aguda (AML)...")
    for _ in range(150):
        wbc = np.random.uniform(25.0, 100.0) # Muy altos
        
        synthetic_patients.append({
            'Sex': np.random.choice([1, 2]), 'Age': np.random.randint(15, 85),
            'WBC': wbc,
            'NEU': wbc * 0.1, # Neutropenia relativa (la máquina cuenta blastos como "otros" o WBC totales)
            'LYM': wbc * 0.2,
            'RBC': np.random.uniform(2.0, 3.0), # Anemia
            'HGB': np.random.uniform(6.0, 9.0),
            'HCT': np.random.uniform(18.0, 27.0),
            'MCV': np.random.uniform(80, 95),
            'MCH': np.random.uniform(27, 32),
            'PLT': np.random.uniform(10, 50), # Trombocitopenia SEVERA
            'Diagnosis': 'Sospecha Leucemia'
        })

    # ==========================================
    # C. FUSIÓN
    # ==========================================
    df_synthetic = pd.DataFrame(synthetic_patients)
    
    # Calcular columnas derivadas si faltan (para mantener consistencia)
    # Si quisieras recalcular porcentajes, hazlo aquí.
    
    # Unir con el dataset real
    # Nota: Aseguramos que tengan las mismas columnas
    # Rellenamos columnas que NHANES tenía y nosotros no generamos con 0 o media (no relevantes para el modelo simplificado)
    common_cols = list(set(df_real.columns) & set(df_synthetic.columns))
    df_final = pd.concat([df_real, df_synthetic], ignore_index=True)
    
    # Guardar
    output_file = "nhanes_augmented_data.csv"
    df_final.to_csv(output_file, index=False)
    
    print("\n📊 Nuevo Balance de Clases:")
    print(df_final['Diagnosis'].value_counts())
    print(f"\n💾 Dataset Aumentado guardado: {output_file}")

if __name__ == "__main__":
    generate_synthetic_data()

🧪 Iniciando Protocolo de Aumento de Datos (Clinical Data Augmentation)...
   -> Sintetizando casos de Anemia Ferropénica...
   -> Sintetizando casos de Anemia Megaloblástica...
   -> Sintetizando simulación de Leucemia Aguda (AML)...

📊 Nuevo Balance de Clases:
Diagnosis
Normal                   5792
Anemia                   1060
Infection                 440
Anemia Ferropénica        300
Thrombocytopenia          235
Anemia Megaloblástica     200
Sospecha Leucemia         150
Name: count, dtype: int64

💾 Dataset Aumentado guardado: nhanes_augmented_data.csv
